# IS537 Data Cleaning Final Project   
 
Candice Chen   
hhchen3@illinois.edu
  
**Dataset source:**  
https://www.kaggle.com/kukuroo3/starbucks-locations-worldwide-2021-version 
  
Starbucks now operates more than 28,289 retail stores in 49 countries. This dataset includes a record for every Starbucks or subsidiary store location currently in operation as of Nov 2021.  

**Topics:**
1. What is the average opening hours in major cities?
2. Ownership type differences?
3. What is the closed days for branches in the same time zone?

**Major cleaning tasks:**
1. Filter countries into subset. We choose The Group of Seven (G7) as our anlyze subjects, which are United States, Canada, France, Germany, Italy, and Japan.
2. Split the "schedule" column into 7 days then fill in opening and closed hours as values.
3. Processing uppercase and lowercase strings in "city" and "countrySubdivisionCode" column.   
4. Split the "olsonTimeZoneId" column into GMT and its region.
5. Merge three columns regarding address. Since there are varions between different countries in the way filling correct format to three address columns, we think it's better to present in only one column with complete information instead of fragmented words.
6. Standardize 'City' column based on each country's format.
7. Drop unrelated columns.

**Teamwork**
* Each member propose some datasets then we chose one of them as our subject after discussion.
* Briefly checked the dataset by columns. Meanwhile, everyone came up some questions that lead to what information we need to analyze and how to get them from this dataset.
* Created subset of the dataset by countries and each member took one of them to clean with their preferreble tools.

**My part**
* This Notebook contains Japan and part of US. We combined cleaned parts in another tool.
* Uesd Python as major cleaning tool. Used Excel for the final step of correcting some typos in Japanese city names.

In [1]:
import pandas as pd
import json
from datetime import datetime

### Read and briefly scan some normal information.

In [2]:
df = pd.read_csv('startbucks_new2.csv')
df.shape

(18985, 17)

In [3]:
df.head()

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
0,1,74256-51879,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1492-110-e-3rd-st-port-an...,48.116035,-123.434818,110 E 3rd St,NaN,NaN,Port Angeles,WA,983623010,-420,Pacific Standard Time,GMT-08:00 America/Los_Angeles
1,2,79652-104215,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1922-2709-e-highway-101-p...,48.105707,-123.379985,2709 E. HIGHWAY 101,NaN,NaN,Port Angeles,WA,983628767,-420,Pacific Standard Time,GMT-08:00 America/Los_Angeles
2,3,9940-96882,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 4:0...",sequim-village-market-place-1095-w-washington-...,48.077861,-123.129862,1095 W. Washington Street,NaN,NaN,Sequim,WA,983823306,-420,Pacific Standard Time,GMT-08:00 America/Los_Angeles
3,4,74255-51779,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-sequim-1448-680-f-w-washington-sequim-...,48.080661,-123.117224,680F W Washington,NaN,NaN,Sequim,WA,983823264,-420,Pacific Standard Time,GMT-08:00 America/Los_Angeles
4,5,64668-298098,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-oak-harbor-402-1450-sw-erie-street-oak...,48.286608,-122.661504,1450 SW Erie Street,NaN,NaN,Oak Harbor,WA,98277,-420,Pacific Standard Time,GMT-08:00 America/Los_Angeles


In [4]:
df.tail()

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
18980,18981,65774-300165,CA,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 8:0...",atlantic-superstore-chain-lake-358-210-chain-l...,44.653480,-63.674914,210 Chain Lake Drive,NaN,NaN,Halifax,NS,B3S 1A2,-180,Atlantic Standard Time,GMT-04:00 America/Halifax
18981,18982,12465-151926,CA,CO,"[{'dayName': 'Today', 'hours': '6:00 AM to 6:0...",86-chain-lake-drive-84-chain-lake-drive-halifa...,44.637634,-63.669763,84 Chain Lake Drive,NaN,NaN,Halifax,NS,B3S 1A2,-180,Atlantic Standard Time,GMT-04:00 America/Halifax
18982,18983,19866-195705,CA,CO,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",535-larry-uteck-blvd-535-larry-uteck-blvd-bedf...,44.701365,-63.678175,535 Larry Uteck Blvd.,NaN,NaN,Bedford,NS,B3M 0E3,-180,Atlantic Standard Time,GMT-04:00 America/Halifax
18983,18984,63618-297759,CA,LS,"[{'dayName': 'Today', 'hours': '7:00 AM to 8:0...",zehrs-windsor-manning-rd-573-400-manning-rd-wi...,42.314574,-82.867337,400 Manning Rd,NaN,NaN,Windsor,ON,N8N 4Z4,-240,Eastern Standard Time,GMT-05:00 America/Toronto
18984,18985,51395-275525,CA,CO,"[{'dayName': 'Today', 'hours': '6:00 AM to 10:...",195-commercial-blvd-195-commercial-blvd-tecums...,42.305531,-82.861515,195 Commercial Blvd,NaN,NaN,Tecumseh,ON,N9K 0A5,-240,Eastern Standard Time,GMT-05:00 America/Toronto


In [5]:
df.columns

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule',
       'slug', 'latitude', 'longitude', 'streetAddressLine1',
       'streetAddressLine2', 'streetAddressLine3', 'city',
       'countrySubdivisionCode', 'postalCode', 'currentTimeOffset',
       'windowsTimeZoneId', 'olsonTimeZoneId'],
      dtype='object')

### 1. Filter by countries and create a subset.

In [6]:
jp = df[df['countryCode'] == "JP"]
jp.tail()

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
16663,16664,26291-235224,JP,CO,NaN,aeon-mall-asahikawa-eki-mae-7-2-5-miyashitador...,43.764304,142.357972,7-2-5 Miyashitadori,NaN,NaN,Asahikawa,1,070-0030,540,Tokyo Standard Time,GMT+09:00 Asia/Tokyo
16664,16665,59730-293558,JP,CO,NaN,asahikawa-kitasaito-1-1-miyamae-2-jo-1-chome-a...,43.761084,142.361242,1-1 Miyamae 2-jo 1-chome,NaN,NaN,Asahikawa,1,078-8392,540,Tokyo Standard Time,GMT+09:00 Asia/Tokyo
16665,16666,62492-295537,JP,CO,NaN,asahikawa-taisetsu-dori-4-491-2-taisetsudori-a...,43.777491,142.385660,4-491-2 Taisetsudori,NaN,NaN,Asahikawa,1,070-0010,540,Tokyo Standard Time,GMT+09:00 Asia/Tokyo
16666,16667,31568-111246,JP,CO,NaN,asahikawa-medical-college-hospital-1-1-midorig...,43.729829,142.384410,1-1 Midorigaoka-higashi 2jo 1-chome,NaN,NaN,Asahikawa,1,078-8510,540,Tokyo Standard Time,GMT+09:00 Asia/Tokyo
16667,16668,47076-251782,JP,CO,NaN,asahikawa-toko-2-6-toko-13-jo-6-chome-asahikaw...,43.742635,142.407044,2-6 Toko 13-jo 6-chome,NaN,NaN,Asahikawa,1,078-8353,540,Tokyo Standard Time,GMT+09:00 Asia/Tokyo


In [7]:
us_2132 = df[df['No.'] <= 2132]
us_2132.tail()

,No.,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,city,countrySubdivisionCode,postalCode,currentTimeOffset,windowsTimeZoneId,olsonTimeZoneId
2127,2128,9709-98003,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 4:0...",auburn-me-mount-auburn-ave-35-mount-auburn-ave...,44.120665,-70.234306,35 Mount Auburn Avenue,NaN,NaN,Auburn,ME,42108521,-240,Eastern Standard Time,GMT-05:00 America/New_York
2128,2129,62844-296537,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 8:0...",shaws-auburn-2550-600-center-street-auburn-me-...,44.123681,-70.227090,600 Center Street,NaN,NaN,Auburn,ME,4210,-240,Eastern Standard Time,GMT-05:00 America/New_York
2129,2130,76677-104732,US,LS,"[{'dayName': 'Today', 'hours': '8:00 AM to 8:0...",target-somersworth-t-2182-11-andrews-rd-somers...,43.232909,-70.881800,11 Andrews Rd,NaN,NaN,Somersworth,NH,38781042,-240,Eastern Standard Time,GMT-05:00 America/New_York
2130,2131,48344-262810,US,CO,"[{'dayName': 'Today', 'hours': '5:30 AM to 6:0...",rochester-marketplace-drive-145-marketplace-dr...,43.332369,-71.007794,"145 Marketplace Drive, Unit 2",Granite Ridge Marketplace,NaN,Rochester,NH,38674383,-240,Eastern Standard Time,GMT-05:00 America/New_York
2131,2132,9991-99148,US,CO,"[{'dayName': 'Today', 'hours': '6:00 AM to 5:0...",dover-nh-webb-place-11-webb-place-dover-nh-038...,43.221388,-70.888269,11 Webb Place,NaN,NaN,Dover,NH,38202403,-240,Eastern Standard Time,GMT-05:00 America/New_York


In [8]:
us_2132.shape

(2132, 17)

In [9]:
combined = pd.concat([us_2132,jp])
combined.shape # make sure the combined dataset with correct numbers of rows

(3797, 17)

In [10]:
combined.columns

Index(['No.', 'storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule',
       'slug', 'latitude', 'longitude', 'streetAddressLine1',
       'streetAddressLine2', 'streetAddressLine3', 'city',
       'countrySubdivisionCode', 'postalCode', 'currentTimeOffset',
       'windowsTimeZoneId', 'olsonTimeZoneId'],
      dtype='object')

### 2. Processing "schedule" column.   

#### Create a subset to process the rows with values in the "schedule" column.

In [11]:
combined_schedule = combined[combined['schedule'].notna()].set_index('No.')['schedule']

#### Use regex to split the column by { } to generate 7 columns of their every day hours.

In [12]:
df_7_days = combined_schedule.str.strip('[]').str.split(r"\s(\{.*?\}),\s+", expand=True)
df_7_days

,0,1,2,3,4,5,6
No.,,,,,,,
1,"{'dayName': 'Today', 'hours': '6:00 AM to 7:00...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 7...","{'dayName': 'Wednesday', 'hours': '6:00 AM to ...","{'dayName': 'Thursday', 'hours': '6:00 AM to 7...","{'dayName': 'Friday', 'hours': '6:00 AM to 7:0...","{'dayName': 'Saturday', 'hours': '6:00 AM to 7...","{'dayName': 'Sunday', 'hours': '6:00 AM to 7:0..."
2,"{'dayName': 'Today', 'hours': '6:00 AM to 7:00...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 7...","{'dayName': 'Wednesday', 'hours': '6:00 AM to ...","{'dayName': 'Thursday', 'hours': '6:00 AM to 7...","{'dayName': 'Friday', 'hours': '6:00 AM to 7:0...","{'dayName': 'Saturday', 'hours': '6:00 AM to 7...","{'dayName': 'Sunday', 'hours': '6:00 AM to 7:0..."
3,"{'dayName': 'Today', 'hours': '5:00 AM to 4:00...","{'dayName': 'Tomorrow', 'hours': '5:00 AM to 8...","{'dayName': 'Wednesday', 'hours': '5:00 AM to ...","{'dayName': 'Thursday', 'hours': '5:00 AM to 8...","{'dayName': 'Friday', 'hours': '5:00 AM to 8:0...","{'dayName': 'Saturday', 'hours': '5:00 AM to 8...","{'dayName': 'Sunday', 'hours': '5:00 AM to 8:0..."
4,"{'dayName': 'Today', 'hours': '6:00 AM to 7:00...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 7...","{'dayName': 'Wednesday', 'hours': '6:00 AM to ...","{'dayName': 'Thursday', 'hours': '6:00 AM to 7...","{'dayName': 'Friday', 'hours': '6:00 AM to 7:0...","{'dayName': 'Saturday', 'hours': '6:00 AM to 7...","{'dayName': 'Sunday', 'hours': '6:00 AM to 7:0..."
5,"{'dayName': 'Today', 'hours': '6:00 AM to 7:00...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 7...","{'dayName': 'Wednesday', 'hours': '6:00 AM to ...","{'dayName': 'Thursday', 'hours': '6:00 AM to 7...","{'dayName': 'Friday', 'hours': '6:00 AM to 7:0...","{'dayName': 'Saturday', 'hours': '6:00 AM to 7...","{'dayName': 'Sunday', 'hours': '6:00 AM to 7:0..."
...,...,...,...,...,...,...,...
15401,"{'dayName': 'Today', 'hours': '9:00 AM to 11:0...","{'dayName': 'Tomorrow', 'hours': '9:00 AM to 1...","{'dayName': 'Thursday', 'hours': '9:00 AM to 1...","{'dayName': 'Friday', 'hours': '9:00 AM to 12:...","{'dayName': 'Saturday', 'hours': '9:00 AM to 1...","{'dayName': 'Sunday', 'hours': '9:00 AM to 12:...","{'dayName': 'Monday', 'hours': '9:00 AM to 11:..."
15461,"{'dayName': 'Today', 'hours': 'Closed', 'open'...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 9...","{'dayName': 'Thursday', 'hours': 'Closed', 'op...","{'dayName': 'Friday', 'hours': 'Closed', 'open...","{'dayName': 'Saturday', 'hours': 'Closed', 'op...","{'dayName': 'Sunday', 'hours': 'Closed', 'open...","{'dayName': 'Monday', 'hours': 'Closed', 'open..."
15504,"{'dayName': 'Today', 'hours': '6:00 AM to 8:00...","{'dayName': 'Tomorrow', 'hours': '6:00 AM to 8...","{'dayName': 'Thursday', 'hours': '6:00 AM to 8...","{'dayName': 'Friday', 'hours': '6:00 AM to 8:0...","{'dayName': 'Saturday', 'hours': '6:00 AM to 8...","{'dayName': 'Sunday', 'hours': '6:00 AM to 8:0...","{'dayName': 'Monday', 'hours': '6:00 AM to 8:0..."


#### Use JSON package in Python to convert strings into dict-like object then create a new dataframe with opening hours in each day.   
Create a list with 8 elements for parsing the "dayName", especially to convert "Today" and "Tomorrow" into corresponding day in a week.

In [13]:
day_list =['Monday', 'Tuesday', 'Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday']

In [14]:
day_open_close = []
for day in day_list[:-2]:
    day_open_close.append(day[0:3]+'_open')
    day_open_close.append(day[0:3]+'_close')

First create an emty dataframe with non-empty rows in 7 days.

In [15]:
df_temp = pd.DataFrame(index=df_7_days.index, columns = day_open_close)
df_temp

,Mon_open,Mon_close,Tue_open,Tue_close,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
No.,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Use for loop to parse each row's value and update corresponding information to new dataframe.

In [16]:
def convert_to_24hour(time: str) -> str:
    if time == 'Closed':
        new = time
    else:
        in_time = datetime.strptime(time, "%I:%M %p")
        new = datetime.strftime(in_time, "%H:%M")    
    return new

In [17]:
for idx in df_7_days.index:
    # get the shedule in a single store and change to JSON format
    single = df_7_days.loc[idx].str.partition(", 'open'", expand=True)[0].str.strip().str.replace("'", '"')+"}"
    
    # use dictionary to store time for 7 days a week
    week = {} 
    
    # convert "Today" and "Tomorrow"
    for i in range(len(single)):
        if day_list[i] in single[6]:
            single[0] = single[0].replace("Today", day_list[i+1])
            single[1] = single[1].replace("Tomorrow", day_list[i+2])
    
    # parse JSON strings in each day
    for day in single:
        # convert JSON format
        info =json.loads(day)
        # create corresponding column name of each day
        day_open = info["dayName"][0:3]+'_open'            
        day_close = info["dayName"][0:3]+'_close'
        # seperate open and close hours
        hr = info['hours'].split(' to ')
        # convert to 24 hours
        df_temp.at[idx,day_open] = convert_to_24hour(hr[0])
        df_temp.at[idx,day_close] = convert_to_24hour(hr[-1])

In [18]:
df_temp.head()

,Mon_open,Mon_close,Tue_open,Tue_close,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
No.,,,,,,,,,,,,,,
1,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
2,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
3,05:00,16:00,05:00,20:00,05:00,20:00,05:00,20:00,05:00,20:00,05:00,20:30,05:00,20:00
4,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
5,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00


In [19]:
df_temp.tail()

,Mon_open,Mon_close,Tue_open,Tue_close,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
No.,,,,,,,,,,,,,,
15401,09:00,23:00,09:00,23:00,09:00,23:00,09:00,23:00,09:00,00:00,09:00,00:00,09:00,00:00
15461,Closed,Closed,Closed,Closed,06:00,21:00,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed
15504,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00,06:00,20:00
15791,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,06:00,21:00,Closed,Closed,Closed,Closed
16431,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,21:00,08:00,16:00,Closed,Closed


#### Merge df_temp dataframe with schedules into the original one by index.

In [20]:
combined_with_schedule = combined.set_index('No.').join(df_temp)
combined_with_schedule.head()

,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,...,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
No.,,,,,,,,,,,,,,,,,,,,,
1,74256-51879,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1492-110-e-3rd-st-port-an...,48.116035,-123.434818,110 E 3rd St,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
2,79652-104215,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1922-2709-e-highway-101-p...,48.105707,-123.379985,2709 E. HIGHWAY 101,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
3,9940-96882,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 4:0...",sequim-village-market-place-1095-w-washington-...,48.077861,-123.129862,1095 W. Washington Street,NaN,NaN,...,05:00,20:00,05:00,20:00,05:00,20:00,05:00,20:30,05:00,20:00
4,74255-51779,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-sequim-1448-680-f-w-washington-sequim-...,48.080661,-123.117224,680F W Washington,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
5,64668-298098,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-oak-harbor-402-1450-sw-erie-street-oak...,48.286608,-122.661504,1450 SW Erie Street,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00


### 3. Processing uppercase and lowercase strings in "city" and "countrySubdivisionCode" column.   

#### Convert "city" to lowercase.

In [21]:
combined_with_schedule['city'].head()

No.
1    Port Angeles
2    Port Angeles
3          Sequim
4          Sequim
5      Oak Harbor
Name: city, dtype: object

In [22]:
combined_with_schedule['city'] = combined_with_schedule['city'].str.lower()

In [23]:
combined_with_schedule['city'].head()

No.
1    port angeles
2    port angeles
3          sequim
4          sequim
5      oak harbor
Name: city, dtype: object

#### Convert "countrySubdivisionCode" to upperercase.

In [24]:
combined_with_schedule['countrySubdivisionCode'].head()

No.
1    WA
2    WA
3    WA
4    WA
5    WA
Name: countrySubdivisionCode, dtype: object

In [25]:
combined_with_schedule['countrySubdivisionCode'] = combined_with_schedule['countrySubdivisionCode'].str.upper()

In [26]:
combined_with_schedule['countrySubdivisionCode'].head()

No.
1    WA
2    WA
3    WA
4    WA
5    WA
Name: countrySubdivisionCode, dtype: object

### 4. Processing "olsonTimeZoneId" column.   

#### Split to "GMT" and "GMT_region".

In [27]:
gmt_region = combined_with_schedule['olsonTimeZoneId'].str.split(expand=True)
gmt_region.columns = ['GMT','GMT_region']
gmt_region.head()

,GMT,GMT_region
No.,,
1,GMT-08:00,America/Los_Angeles
2,GMT-08:00,America/Los_Angeles
3,GMT-08:00,America/Los_Angeles
4,GMT-08:00,America/Los_Angeles
5,GMT-08:00,America/Los_Angeles


In [28]:
combined_with_schedule = combined_with_schedule.join(gmt_region)
combined_with_schedule.head()

,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,...,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close,GMT,GMT_region
No.,,,,,,,,,,,,,,,,,,,,,
1,74256-51879,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1492-110-e-3rd-st-port-an...,48.116035,-123.434818,110 E 3rd St,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles
2,79652-104215,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1922-2709-e-highway-101-p...,48.105707,-123.379985,2709 E. HIGHWAY 101,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles
3,9940-96882,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 4:0...",sequim-village-market-place-1095-w-washington-...,48.077861,-123.129862,1095 W. Washington Street,NaN,NaN,...,05:00,20:00,05:00,20:00,05:00,20:30,05:00,20:00,GMT-08:00,America/Los_Angeles
4,74255-51779,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-sequim-1448-680-f-w-washington-sequim-...,48.080661,-123.117224,680F W Washington,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles
5,64668-298098,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-oak-harbor-402-1450-sw-erie-street-oak...,48.286608,-122.661504,1450 SW Erie Street,NaN,NaN,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles


### 5. Merge three columns regarding address. 

#### Split to .

In [29]:
combined_with_schedule.columns

Index(['storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule', 'slug',
       'latitude', 'longitude', 'streetAddressLine1', 'streetAddressLine2',
       'streetAddressLine3', 'city', 'countrySubdivisionCode', 'postalCode',
       'currentTimeOffset', 'windowsTimeZoneId', 'olsonTimeZoneId', 'Mon_open',
       'Mon_close', 'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close',
       'Thu_open', 'Thu_close', 'Fri_open', 'Fri_close', 'Sat_open',
       'Sat_close', 'Sun_open', 'Sun_close', 'GMT', 'GMT_region'],
      dtype='object')

#### Set filters by null values in three address columns.

In [30]:
address = ['streetAddressLine1','streetAddressLine2', 'streetAddressLine3']
filter_1 = combined_with_schedule['streetAddressLine1'].notna()
filter_2 = combined_with_schedule['streetAddressLine2'].notna()
filter_3 = combined_with_schedule['streetAddressLine3'].notna()

#### If at least two of the address columns have values, combine with a delimter "|" between them. 

In [31]:
a = combined_with_schedule[filter_1 & filter_2 & filter_3][address]
a['address'] = a[address[0]] + "|" + a[address[1]] + "|" + a[address[2]]
a

,streetAddressLine1,streetAddressLine2,streetAddressLine3,address
No.,,,,
43,9660 Transit Road,101,Tops Transit Commons Shopping Ctr.,9660 Transit Road|101|Tops Transit Commons Sho...
56,2223 S. Monaco Parkway,G100,Villa Monaco,2223 S. Monaco Parkway|G100|Villa Monaco
60,8000 E Belleview,Suite B10,Denver Tech Center,8000 E Belleview|Suite B10|Denver Tech Center
108,1800 E Franklin Street,Suite B,Eastgate Shopping Center,1800 E Franklin Street|Suite B|Eastgate Shoppi...
128,7854 Alexander Promenade Pl.,"Bldg. H, Ste. 1",Alexander Place,"7854 Alexander Promenade Pl.|Bldg. H, Ste. 1|A..."
...,...,...,...,...
2017,434 Gammon Place,3,Madison Gammon Place,434 Gammon Place|3|Madison Gammon Place
2033,2 Theatre Square,106,Orinda Theatre Square,2 Theatre Square|106|Orinda Theatre Square
2053,1544 Palos Verdes Mall,Suite 44,Palos Verdes Mall,1544 Palos Verdes Mall|Suite 44|Palos Verdes Mall


In [32]:
b = combined_with_schedule[filter_1 & ~filter_2 & filter_3][address]
b['address'] = b[address[0]] + "|" + b[address[2]]
b

,streetAddressLine1,streetAddressLine2,streetAddressLine3,address
No.,,,,
10,1720 Commercial Avenue,NaN,Fidalgo Square,1720 Commercial Avenue|Fidalgo Square
37,520 Lee Entrance,NaN,The Commons - Buffalo University,520 Lee Entrance|The Commons - Buffalo University
51,2700 S. Colorado Blvd.,NaN,University Hills Mall,2700 S. Colorado Blvd.|University Hills Mall
83,1402 S. Havana St.,NaN,Gardens in Havana,1402 S. Havana St.|Gardens in Havana
111,6813 Fayetteville Road,NaN,Renaissance at Southpoint,6813 Fayetteville Road|Renaissance at Southpoint
...,...,...,...,...
2074,2922 North Main Street,NaN,The Walden Center,2922 North Main Street|The Walden Center
2076,3060A E 9th St,NaN,Fruitvale Station,3060A E 9th St|Fruitvale Station
2090,7521 S University Blvd,NaN,Cherrywood Shopping Center,7521 S University Blvd|Cherrywood Shopping Center


In [33]:
c = combined_with_schedule[filter_1 & filter_2 & ~filter_3][address]
c['address'] = c[address[0]] + "|" + c[address[1]]
c

,streetAddressLine1,streetAddressLine2,streetAddressLine3,address
No.,,,,
6,31270 SR 20,Unit A-1,NaN,31270 SR 20|Unit A-1
9,32650 SR 20,A-107,NaN,32650 SR 20|A-107
17,8082 Guide Meridian Ave,101,NaN,8082 Guide Meridian Ave|101
19,4303 Guide Meridian,101,NaN,4303 Guide Meridian|101
26,6690 Niagara Falls Boulevard,100,NaN,6690 Niagara Falls Boulevard|100
...,...,...,...,...
15666,2-20 Kita7-jo Higashi 9-chome,Higashi-ku,NaN,2-20 Kita7-jo Higashi 9-chome|Higashi-ku
15674,1-1 Higashinaebo 2-jo,3-chome Higashi-ku,NaN,1-1 Higashinaebo 2-jo|3-chome Higashi-ku
15676,1-29 Tsukisamu Higashi 3-jo 11-chome,Toyohira-ku,NaN,1-29 Tsukisamu Higashi 3-jo 11-chome|Toyohira-ku


#### Get the rows that only have values in first column.

In [34]:
d = combined_with_schedule[filter_1 & ~filter_2 & ~filter_3][address]
d['address'] = d[address[0]]
d

,streetAddressLine1,streetAddressLine2,streetAddressLine3,address
No.,,,,
1,110 E 3rd St,NaN,NaN,110 E 3rd St
2,2709 E. HIGHWAY 101,NaN,NaN,2709 E. HIGHWAY 101
3,1095 W. Washington Street,NaN,NaN,1095 W. Washington Street
4,680F W Washington,NaN,NaN,680F W Washington
5,1450 SW Erie Street,NaN,NaN,1450 SW Erie Street
...,...,...,...,...
16664,7-2-5 Miyashitadori,NaN,NaN,7-2-5 Miyashitadori
16665,1-1 Miyamae 2-jo 1-chome,NaN,NaN,1-1 Miyamae 2-jo 1-chome
16666,4-491-2 Taisetsudori,NaN,NaN,4-491-2 Taisetsudori


#### Merge all rows with new values of the combined addresses.

In [35]:
frames = [a, b, c, d]
adrress = pd.concat(frames)
adrress

,streetAddressLine1,streetAddressLine2,streetAddressLine3,address
No.,,,,
43,9660 Transit Road,101,Tops Transit Commons Shopping Ctr.,9660 Transit Road|101|Tops Transit Commons Sho...
56,2223 S. Monaco Parkway,G100,Villa Monaco,2223 S. Monaco Parkway|G100|Villa Monaco
60,8000 E Belleview,Suite B10,Denver Tech Center,8000 E Belleview|Suite B10|Denver Tech Center
108,1800 E Franklin Street,Suite B,Eastgate Shopping Center,1800 E Franklin Street|Suite B|Eastgate Shoppi...
128,7854 Alexander Promenade Pl.,"Bldg. H, Ste. 1",Alexander Place,"7854 Alexander Promenade Pl.|Bldg. H, Ste. 1|A..."
...,...,...,...,...
16664,7-2-5 Miyashitadori,NaN,NaN,7-2-5 Miyashitadori
16665,1-1 Miyamae 2-jo 1-chome,NaN,NaN,1-1 Miyamae 2-jo 1-chome
16666,4-491-2 Taisetsudori,NaN,NaN,4-491-2 Taisetsudori


In [36]:
combined_with_address = combined_with_schedule.join(adrress['address'])
combined_with_address.head()

,storeNumber,countryCode,ownershipTypeCode,schedule,slug,latitude,longitude,streetAddressLine1,streetAddressLine2,streetAddressLine3,...,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close,GMT,GMT_region,address
No.,,,,,,,,,,,,,,,,,,,,,
1,74256-51879,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1492-110-e-3rd-st-port-an...,48.116035,-123.434818,110 E 3rd St,NaN,NaN,...,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles,110 E 3rd St
2,79652-104215,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-port-angeles-1922-2709-e-highway-101-p...,48.105707,-123.379985,2709 E. HIGHWAY 101,NaN,NaN,...,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles,2709 E. HIGHWAY 101
3,9940-96882,US,CO,"[{'dayName': 'Today', 'hours': '5:00 AM to 4:0...",sequim-village-market-place-1095-w-washington-...,48.077861,-123.129862,1095 W. Washington Street,NaN,NaN,...,20:00,05:00,20:00,05:00,20:30,05:00,20:00,GMT-08:00,America/Los_Angeles,1095 W. Washington Street
4,74255-51779,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-sequim-1448-680-f-w-washington-sequim-...,48.080661,-123.117224,680F W Washington,NaN,NaN,...,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles,680F W Washington
5,64668-298098,US,LS,"[{'dayName': 'Today', 'hours': '6:00 AM to 7:0...",safeway-oak-harbor-402-1450-sw-erie-street-oak...,48.286608,-122.661504,1450 SW Erie Street,NaN,NaN,...,19:00,06:00,19:00,06:00,19:00,06:00,19:00,GMT-08:00,America/Los_Angeles,1450 SW Erie Street


In [37]:
combined_with_address.columns

Index(['storeNumber', 'countryCode', 'ownershipTypeCode', 'schedule', 'slug',
       'latitude', 'longitude', 'streetAddressLine1', 'streetAddressLine2',
       'streetAddressLine3', 'city', 'countrySubdivisionCode', 'postalCode',
       'currentTimeOffset', 'windowsTimeZoneId', 'olsonTimeZoneId', 'Mon_open',
       'Mon_close', 'Tue_open', 'Tue_close', 'Wed_open', 'Wed_close',
       'Thu_open', 'Thu_close', 'Fri_open', 'Fri_close', 'Sat_open',
       'Sat_close', 'Sun_open', 'Sun_close', 'GMT', 'GMT_region', 'address'],
      dtype='object')

### 6. Drop unrelated columns.   

In [38]:
drop_columns = ['schedule','slug', 'streetAddressLine1','streetAddressLine2', 'streetAddressLine3', 
                'currentTimeOffset', 'olsonTimeZoneId']

In [39]:
df_cleaned = combined_with_address.drop(columns = drop_columns)

In [40]:
len(combined_with_address.columns) > len(df_cleaned.columns)

True

In [41]:
df_cleaned.columns

Index(['storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude',
       'longitude', 'city', 'countrySubdivisionCode', 'postalCode',
       'windowsTimeZoneId', 'Mon_open', 'Mon_close', 'Tue_open', 'Tue_close',
       'Wed_open', 'Wed_close', 'Thu_open', 'Thu_close', 'Fri_open',
       'Fri_close', 'Sat_open', 'Sat_close', 'Sun_open', 'Sun_close', 'GMT',
       'GMT_region', 'address'],
      dtype='object')

#### Sort the columns into the order we discussed.

In [42]:
new_column_order = ['storeNumber', 'countryCode', 'ownershipTypeCode', 'latitude', 'longitude', 
                    'address','city', 'countrySubdivisionCode', 'postalCode','windowsTimeZoneId',
                    'GMT', 'GMT_region', 'Mon_open', 'Mon_close', 'Tue_open', 'Tue_close','Wed_open', 
                    'Wed_close', 'Thu_open', 'Thu_close', 'Fri_open','Fri_close', 'Sat_open', 'Sat_close', 
                    'Sun_open', 'Sun_close']
len(new_column_order)

26

In [43]:
len(new_column_order) == len(df_cleaned.columns)

True

In [44]:
df_cleaned_sorted = df_cleaned[new_column_order]
df_cleaned_sorted

,storeNumber,countryCode,ownershipTypeCode,latitude,longitude,address,city,countrySubdivisionCode,postalCode,windowsTimeZoneId,...,Wed_open,Wed_close,Thu_open,Thu_close,Fri_open,Fri_close,Sat_open,Sat_close,Sun_open,Sun_close
No.,,,,,,,,,,,,,,,,,,,,,
1,74256-51879,US,LS,48.116035,-123.434818,110 E 3rd St,port angeles,WA,983623010,Pacific Standard Time,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
2,79652-104215,US,LS,48.105707,-123.379985,2709 E. HIGHWAY 101,port angeles,WA,983628767,Pacific Standard Time,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
3,9940-96882,US,CO,48.077861,-123.129862,1095 W. Washington Street,sequim,WA,983823306,Pacific Standard Time,...,05:00,20:00,05:00,20:00,05:00,20:00,05:00,20:30,05:00,20:00
4,74255-51779,US,LS,48.080661,-123.117224,680F W Washington,sequim,WA,983823264,Pacific Standard Time,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
5,64668-298098,US,LS,48.286608,-122.661504,1450 SW Erie Street,oak harbor,WA,98277,Pacific Standard Time,...,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00,06:00,19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16664,26291-235224,JP,CO,43.764304,142.357972,7-2-5 Miyashitadori,asahikawa,1,070-0030,Tokyo Standard Time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16665,59730-293558,JP,CO,43.761084,142.361242,1-1 Miyamae 2-jo 1-chome,asahikawa,1,078-8392,Tokyo Standard Time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16666,62492-295537,JP,CO,43.777491,142.385660,4-491-2 Taisetsudori,asahikawa,1,070-0010,Tokyo Standard Time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 7. Output a csv file then use Excel to correct typos.
Since city names in Japan are translated to English by pronunciation, we found some tiny difference by visual observation. We kept most of them in the original version due to the difficulties to validate. We only correct obvious typos by viewing the vaues in Excel.  
  
**Here are the changes we made:**  
wakayama -> wakayama  
utsunomi -> utsunomiya  
fukukoka -> fukuoka  
gotenba -> gotemba  
kitakyusyu -> kitakyushu

In [45]:
df_cleaned_sorted.to_csv('startbucks_jp_us2132_cleaned.csv', index=True)